<a href="https://colab.research.google.com/github/SamanthaMcKay/Deep-Learning-Challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME","SPECIAL_CONSIDERATIONS"], axis=1)

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts=application_df["APPLICATION_TYPE"].value_counts()

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Set a cutoff value based on your criteria
cutoff_value = 200
# Create a list of application types to be replaced with 'Other'
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_type_counts=application_df["CLASSIFICATION"].value_counts()
print(classification_type_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# Look at CLASSIFICATION value counts >1
for index, count in classification_type_counts.iteritems():
    if count > 1:
        print(f"{index}: {count}")
    else:
        continue

C1000: 17326
C2000: 6074
C1200: 4837
C3000: 1918
C2100: 1883
C7000: 777
C1700: 287
C4000: 194
C5000: 116
C1270: 114
C2700: 104
C2800: 95
C7100: 75
C1300: 58
C1280: 50
C1230: 36
C1400: 34
C7200: 32
C2300: 32
C1240: 30
C8000: 20
C7120: 18
C1500: 16
C1800: 15
C6000: 15
C1250: 14
C8200: 11
C1238: 10
C1278: 10
C1235: 9
C1237: 9
C7210: 7
C2400: 6
C1720: 6
C4100: 6
C1257: 5
C1600: 5
C1260: 3
C2710: 3
C0: 3
C3200: 2
C1234: 2
C1246: 2
C1267: 2
C1256: 2


<ipython-input-7-e82ee09e6418>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, count in classification_type_counts.iteritems():


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 200
# Create a list of classification types to be replaced with 'Other'
classifications_to_replace = classification_type_counts[classification_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
# Specify the columns you want to one-hot encode
columns_to_encode = ['APPLICATION_TYPE', 'INCOME_AMT','CLASSIFICATION','AFFILIATION',"ORGANIZATION","USE_CASE"]

# Use get_dummies to convert the specified columns into dummy variables
application_df_dummies = pd.get_dummies(application_df[columns_to_encode], prefix=columns_to_encode, drop_first=True)

# Concatenate the dummy variables with the original DataFrame
application_df = pd.concat([application_df, application_df_dummies], axis=1)

# Drop the original categorical columns if needed
application_df = application_df.drop(columns_to_encode, axis=1)

# Print the resulting DataFrame
print(application_df)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_T10  \
0           1      5000              1                     1   
1           1    108590              1                     0   
2           1      5000              0                     0   
3           1      6692              1                     0   
4           1    142590              1                     0   
...       ...       ...            ...                   ...   
34294       1      5000              0                     0   
34295       1      5000              0                     0   
34296       1      5000              0                     0   
34297       1      5000              1                     0   
34298       1  36500179              0                     0   

       APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                         0                    0                    0   
1                         0                    1                    0   
2           

In [10]:
application_df.head(10)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
5,1,5000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
6,1,31452,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
7,1,7508025,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,1,94389,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
9,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
print(application_df.nunique())

STATUS                          2
ASK_AMT                      8747
IS_SUCCESSFUL                   2
APPLICATION_TYPE_T10            2
APPLICATION_TYPE_T19            2
APPLICATION_TYPE_T3             2
APPLICATION_TYPE_T4             2
APPLICATION_TYPE_T5             2
APPLICATION_TYPE_T6             2
APPLICATION_TYPE_T7             2
APPLICATION_TYPE_T8             2
INCOME_AMT_1-9999               2
INCOME_AMT_10000-24999          2
INCOME_AMT_100000-499999        2
INCOME_AMT_10M-50M              2
INCOME_AMT_1M-5M                2
INCOME_AMT_25000-99999          2
INCOME_AMT_50M+                 2
INCOME_AMT_5M-10M               2
CLASSIFICATION_C1200            2
CLASSIFICATION_C1700            2
CLASSIFICATION_C2000            2
CLASSIFICATION_C2100            2
CLASSIFICATION_C3000            2
CLASSIFICATION_C7000            2
CLASSIFICATION_Other            2
AFFILIATION_Family/Parent       2
AFFILIATION_Independent         2
AFFILIATION_National            2
AFFILIATION_Ot

In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
print(X_train_scaled)

[[ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982  0.47226103
  -0.44744306]
 [ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982 -2.11747306
   2.23492125]
 [ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982  0.47226103
  -0.44744306]
 ...
 [ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982  0.47226103
  -0.44744306]
 [ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982  0.47226103
  -0.44744306]
 [ 0.01079982 -0.02957055 -0.12358845 ... -0.01079982  0.47226103
  -0.44744306]]


## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=74, activation="tanh", input_dim=37))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=74, activation="tanh"))

# As part of optimization, third hidden layer
nn_model.add(tf.keras.layers.Dense(units=74, activation="tanh"))



# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 74)                2812      
                                                                 
 dense_9 (Dense)             (None, 74)                5550      
                                                                 
 dense_10 (Dense)            (None, 74)                5550      
                                                                 
 dense_11 (Dense)            (None, 1)                 75        
                                                                 
Total params: 13987 (54.64 KB)
Trainable params: 13987 (54.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5737 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5586 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5550 - accuracy: 0.7272
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7292
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7301
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5572 - accuracy: 0.7292 - 526ms/epoch - 2ms/step
Loss: 0.5572447180747986, Accuracy: 0.7292128205299377


In [25]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
